# GREAT CARIA - Formal Fragility Theory

## Framework:
1. **F_t Formal Definition**: Latent fragility as interaction of signals
2. **Temporal Decomposition**: Slow/Medium/Fast bands
3. **Extended Crisis Catalog**: 10+ events (not just 2)
4. **Bifurcation Dynamics**: Cusp catastrophe, saddle-node, dynamic coupling

In [ ]:
!pip install PyWavelets networkx scikit-learn -q

import pandas as pd
import numpy as np
from scipy import stats, signal, optimize
from scipy.ndimage import gaussian_filter1d
from scipy.linalg import eigh
import pywt
import networkx as nx
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

MARKET_PATH = '/content/drive/MyDrive/CARIA/data/raw/yahoo_market.parquet'
df = pd.read_parquet(MARKET_PATH)
COUNTRIES = ['USA', 'CHN', 'JPN', 'DEU', 'GBR', 'FRA', 'BRA', 'MEX', 'KOR', 'AUS', 'IND', 'ZAF']
print(f'Data: {df.shape}, Range: {df.index.min().date()} to {df.index.max().date()}')

---
# CRISIS CATALOG (Extended)

| Event | Date | Type |
|-------|------|------|
| DotCom Peak | 2000-03-10 | Bubble |
| 9/11 | 2001-09-11 | Shock |
| Lehman | 2008-09-15 | Systemic |
| Flash Crash | 2010-05-06 | Liquidity |
| Euro Crisis | 2011-08-05 | Sovereign |
| Taper Tantrum | 2013-05-22 | Policy |
| China Crash | 2015-08-24 | Emerging |
| Brexit | 2016-06-24 | Political |
| Volmageddon | 2018-02-05 | Vol |
| Repo Crisis | 2019-09-17 | Liquidity |
| COVID | 2020-03-11 | Pandemic |
| Gilt Crisis | 2022-09-23 | Sovereign |
| SVB | 2023-03-10 | Banking |

In [ ]:
# === EXTENDED CRISIS CATALOG ===
CRISES = {
    'DotCom': pd.Timestamp('2000-03-10'),
    '9/11': pd.Timestamp('2001-09-11'),
    'Lehman': pd.Timestamp('2008-09-15'),
    'Flash_Crash': pd.Timestamp('2010-05-06'),
    'Euro_Crisis': pd.Timestamp('2011-08-05'),
    'Taper_Tantrum': pd.Timestamp('2013-05-22'),
    'China_Crash': pd.Timestamp('2015-08-24'),
    'Brexit': pd.Timestamp('2016-06-24'),
    'Volmageddon': pd.Timestamp('2018-02-05'),
    'Repo_Crisis': pd.Timestamp('2019-09-17'),
    'COVID': pd.Timestamp('2020-03-11'),
    'Gilt_Crisis': pd.Timestamp('2022-09-23'),
    'SVB': pd.Timestamp('2023-03-10')
}

# Filter to available data
data_start = df.index.min()
CRISES = {k: v for k, v in CRISES.items() if v >= data_start}
print(f'Crises in data range: {len(CRISES)}')
for name, date in CRISES.items():
    print(f'  {name}: {date.date()}')

In [ ]:
# === BASE SIGNALS ===
idx_cols = [f'{c}_index' for c in COUNTRIES if f'{c}_index' in df.columns]
ret = df[idx_cols].pct_change().dropna()
ret.columns = [c.replace('_index', '') for c in ret.columns]

# 1. Crisis Factor (CF)
def compute_cf(r, w=20):
    cf = []
    for i in range(w, len(r)):
        wr = r.iloc[i-w:i]
        c = wr.corr().values
        ac = (c.sum() - len(c)) / (len(c) * (len(c) - 1))
        cf.append(ac * wr.std().mean() * 100)
    return pd.Series(cf, index=r.index[w:])

CF = compute_cf(ret)

# 2. Synchronization (Kuramoto)
def extract_phase(series):
    detrended = series - gaussian_filter1d(series.values, sigma=60)
    return np.angle(signal.hilbert(detrended))

phases = pd.DataFrame({c: extract_phase(ret[c].fillna(0)) for c in ret.columns}, index=ret.index)

def kuramoto_order(phases, window=60):
    r = [np.abs(np.exp(1j * phases.iloc[i].values).mean()) for i in range(window, len(phases))]
    return pd.Series(r, index=phases.index[window:])

SYNC = kuramoto_order(phases)

# 3. EWS (ACF, Var, Skew)
def compute_ews(series, window=120):
    return pd.DataFrame({
        'acf1': series.rolling(window).apply(lambda x: x.autocorr(1), raw=False),
        'var': series.rolling(window).var(),
        'skew': series.rolling(window).skew()
    })

EWS = compute_ews(CF)

# 4. Network curvature proxy (avg correlation)
def rolling_avg_corr(r, window=60):
    corrs = []
    for i in range(window, len(r)):
        c = r.iloc[i-window:i].corr().values
        avg = (c.sum() - len(c)) / (len(c) * (len(c) - 1))
        corrs.append(avg)
    return pd.Series(corrs, index=r.index[window:])

CURV = 1 - rolling_avg_corr(ret, 60)  # Low correlation = low curvature = fragile

print(f'Signals computed: CF={len(CF)}, SYNC={len(SYNC)}, EWS={len(EWS.dropna())}')

---
# PART 1: FORMAL FRAGILITY DEFINITION

$$F_t = f(CF_t, S_t, ACF_t, Var_t, Skew_t, Topology_t)$$

Using **Factor Analysis** to extract a latent fragility variable

In [ ]:
# === 1A: Align all signals ===
common_idx = CF.index
for sig in [SYNC, EWS['acf1'], EWS['var'], EWS['skew'], CURV]:
    common_idx = common_idx.intersection(sig.dropna().index)

signals_df = pd.DataFrame({
    'cf': CF.loc[common_idx],
    'sync': SYNC.loc[common_idx],
    'acf1': EWS['acf1'].loc[common_idx],
    'var': EWS['var'].loc[common_idx],
    'skew': EWS['skew'].abs().loc[common_idx],
    'curv': CURV.loc[common_idx]
}).dropna()

print(f'Aligned signals: {signals_df.shape}')

In [ ]:
# === 1B: Factor Analysis for latent F_t ===
print('=== Factor Analysis for Latent Fragility ===')

# Standardize
scaler = StandardScaler()
X = scaler.fit_transform(signals_df)

# Factor Analysis with 1 latent factor
fa = FactorAnalysis(n_components=1, random_state=42)
F_latent = fa.fit_transform(X).flatten()
F_t = pd.Series(F_latent, index=signals_df.index, name='F_t')

# Loadings show how each signal contributes
loadings = pd.DataFrame({
    'signal': signals_df.columns,
    'loading': fa.components_[0]
}).sort_values('loading', ascending=False)

print('\nFactor Loadings (contribution to latent fragility):')
print(loadings.to_string(index=False))

# Explained variance
pca = PCA(n_components=1)
pca.fit(X)
print(f'\nVariance explained by F_t: {pca.explained_variance_ratio_[0]:.1%}')

In [ ]:
# === 1C: Validate F_t against crises ===

fig, ax = plt.subplots(figsize=(14, 5))
ax.fill_between(F_t.index, F_t.values, alpha=0.3, color='red')
ax.plot(F_t.index, F_t.values, 'r-', linewidth=0.5)
ax.axhline(F_t.quantile(0.9), color='orange', linestyle='--', label='90th pct')

for name, date in CRISES.items():
    if date in F_t.index or (date > F_t.index.min() and date < F_t.index.max()):
        ax.axvline(date, color='blue', alpha=0.5, linestyle=':')
        ax.text(date, ax.get_ylim()[1], name, rotation=90, fontsize=8, va='top')

ax.set_title('Latent Fragility F_t (Factor Analysis)')
ax.legend()
plt.tight_layout()
plt.show()

---
# PART 2: TEMPORAL DECOMPOSITION

Decompose into:
- **Slow Band** (>60 days): Structural/Cycle
- **Medium Band** (10-60 days): Resonance/Systemic
- **Fast Band** (<10 days): Critical noise

In [ ]:
# === 2A: Wavelet decomposition into bands ===
print('=== Temporal Band Decomposition ===')

def decompose_bands(series, wavelet='db4'):
    """Decompose into slow/medium/fast bands"""
    coeffs = pywt.wavedec(series.values, wavelet, level=6)
    
    # Fast (levels 0-2): ~1-8 days
    fast_coeffs = [np.zeros_like(c) for c in coeffs]
    for i in range(min(3, len(coeffs))):
        fast_coeffs[i] = coeffs[i]
    fast = pywt.waverec(fast_coeffs, wavelet)[:len(series)]
    
    # Medium (levels 3-4): ~8-64 days
    med_coeffs = [np.zeros_like(c) for c in coeffs]
    for i in range(3, min(5, len(coeffs))):
        med_coeffs[i] = coeffs[i]
    medium = pywt.waverec(med_coeffs, wavelet)[:len(series)]
    
    # Slow (levels 5+): >64 days
    slow_coeffs = [np.zeros_like(c) for c in coeffs]
    for i in range(5, len(coeffs)):
        slow_coeffs[i] = coeffs[i]
    slow = pywt.waverec(slow_coeffs, wavelet)[:len(series)]
    
    return {
        'fast': pd.Series(fast, index=series.index),
        'medium': pd.Series(medium, index=series.index),
        'slow': pd.Series(slow, index=series.index)
    }

# Decompose F_t
F_bands = decompose_bands(F_t)
print(f'Decomposition complete')

In [ ]:
# === 2B: Visualize bands ===

fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

axes[0].plot(F_t.index, F_t.values, 'k-', alpha=0.7)
axes[0].set_ylabel('F_t (full)')
axes[0].set_title('Temporal Decomposition of Latent Fragility')

axes[1].plot(F_bands['slow'].index, F_bands['slow'].values, 'b-', linewidth=2)
axes[1].set_ylabel('Slow (>60d)')
axes[1].set_title('Structural/Cycle Component')

axes[2].plot(F_bands['medium'].index, F_bands['medium'].values, 'orange', linewidth=1)
axes[2].set_ylabel('Medium (10-60d)')
axes[2].set_title('Systemic Resonance Component')

axes[3].plot(F_bands['fast'].index, F_bands['fast'].values, 'r-', alpha=0.7, linewidth=0.5)
axes[3].set_ylabel('Fast (<10d)')
axes[3].set_title('Critical Noise Component')

for ax in axes:
    for name, date in CRISES.items():
        if date > F_t.index.min() and date < F_t.index.max():
            ax.axvline(date, color='red', alpha=0.3, linestyle=':')

plt.tight_layout()
plt.show()

In [ ]:
# === 2C: Which band leads? ===
print('\n=== Band Lead-Lag Analysis ===')

def compute_lead(band, crisis_date, threshold_pct=0.8):
    """How many days before crisis did band cross threshold?"""
    threshold = band.quantile(threshold_pct)
    pre = band[(band.index < crisis_date) & (band.index > crisis_date - pd.Timedelta(days=180))]
    crossings = pre[pre > threshold]
    if len(crossings) > 0:
        return (crisis_date - crossings.index[0]).days
    return 0

band_leads = {band: [] for band in ['slow', 'medium', 'fast']}

for crisis_name, crisis_date in CRISES.items():
    if crisis_date < F_t.index.min() or crisis_date > F_t.index.max():
        continue
    for band in ['slow', 'medium', 'fast']:
        lead = compute_lead(F_bands[band], crisis_date)
        band_leads[band].append(lead)

print('Average lead time by band:')
for band, leads in band_leads.items():
    avg = np.mean(leads) if leads else 0
    print(f'  {band:8s}: {avg:.0f} days')

---
# PART 3: BIFURCATION DYNAMICS

## Cusp Catastrophe Model:
$$V(x) = \frac{x^4}{4} + \frac{a \cdot x^2}{2} + b \cdot x$$

Where:
- $x$ = system state (F_t)
- $a$ = asymmetry parameter
- $b$ = bifurcation parameter

In [ ]:
# === 3A: Cusp catastrophe fitting ===
print('=== Cusp Catastrophe Analysis ===')

def cusp_potential(x, a, b):
    """Cusp catastrophe potential V(x) = x^4/4 + a*x^2/2 + b*x"""
    return x**4/4 + a*x**2/2 + b*x

def cusp_gradient(x, a, b):
    """dV/dx = x^3 + a*x + b"""
    return x**3 + a*x + b

def fit_cusp_params(series, window=120):
    """Estimate cusp parameters from time series"""
    a_params = []
    b_params = []
    bimodality = []
    dates = []
    
    for i in range(window, len(series), 10):
        window_data = series.iloc[i-window:i].values
        
        # Estimate a (related to skewness)
        skew = stats.skew(window_data)
        
        # Estimate b (related to kurtosis)
        kurt = stats.kurtosis(window_data)
        
        # Bimodality coefficient
        bimod = (skew**2 + 1) / (kurt + 3 + 1e-8)
        
        # Map to cusp parameters (heuristic)
        a = -np.sign(kurt) * np.abs(kurt) / 10  # Negative a = two minima
        b = skew / 5
        
        a_params.append(a)
        b_params.append(b)
        bimodality.append(bimod)
        dates.append(series.index[i])
    
    return pd.DataFrame({
        'a': a_params,
        'b': b_params,
        'bimodality': bimodality
    }, index=dates)

cusp_params = fit_cusp_params(F_t)
print(f'Cusp parameters computed: {len(cusp_params)} windows')

In [ ]:
# === 3B: Bifurcation diagram ===

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

axes[0].plot(cusp_params.index, cusp_params['a'], 'b-')
axes[0].axhline(0, color='gray', linestyle='--')
axes[0].set_ylabel('a (bifurcation)')
axes[0].set_title('Cusp Catastrophe Parameters')
# Negative a = unstable (two attractors possible)
axes[0].fill_between(cusp_params.index, cusp_params['a'], 0, 
                     where=cusp_params['a'] < 0, alpha=0.3, color='red')

axes[1].plot(cusp_params.index, cusp_params['b'], 'orange')
axes[1].axhline(0, color='gray', linestyle='--')
axes[1].set_ylabel('b (asymmetry)')

axes[2].plot(cusp_params.index, cusp_params['bimodality'], 'purple')
axes[2].axhline(0.555, color='red', linestyle='--', label='Bimodal threshold')
axes[2].set_ylabel('Bimodality')
axes[2].legend()

for ax in axes:
    for name, date in CRISES.items():
        if date > cusp_params.index.min() and date < cusp_params.index.max():
            ax.axvline(date, color='red', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# === 3C: Saddle-node dynamics (proximity to bifurcation) ===
print('\n=== Saddle-Node Dynamics ===')

def saddle_node_distance(series, window=60):
    """Distance to saddle-node bifurcation
    
    Near bifurcation:
    - Variance increases
    - ACF(1) → 1 (critical slowing down)
    - Skewness changes sign
    """
    distances = []
    for i in range(window, len(series)):
        w = series.iloc[i-window:i]
        
        # Normalized indicators
        acf1 = w.autocorr(1)
        var_norm = w.var() / series.var()  # Relative variance
        
        # Distance to bifurcation (lower = closer)
        dist = (1 - acf1) * (1 / (var_norm + 0.01))
        distances.append(dist)
    
    return pd.Series(distances, index=series.index[window:])

saddle_dist = saddle_node_distance(F_t)

# Lower distance = closer to bifurcation
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(saddle_dist.index, saddle_dist.values, 'g-', alpha=0.7)
ax.set_ylabel('Distance to Bifurcation')
ax.set_title('Saddle-Node Bifurcation Proximity (lower = closer to crisis)')
ax.invert_yaxis()  # Invert so "closer to crisis" is up

for name, date in CRISES.items():
    if date > saddle_dist.index.min() and date < saddle_dist.index.max():
        ax.axvline(date, color='red', alpha=0.5)

plt.show()

---
# PART 4: UNIFIED FRAGILITY TENSOR

Combine all components into a single framework

In [ ]:
# === 4A: Construct Fragility Tensor ===
print('=== Constructing Fragility Tensor ===')

# Align all components
tensor_idx = F_t.index.intersection(cusp_params.index).intersection(saddle_dist.index)

fragility_tensor = pd.DataFrame({
    # Latent state
    'F_t': F_t.loc[tensor_idx],
    
    # Temporal components
    'F_slow': F_bands['slow'].loc[tensor_idx],
    'F_medium': F_bands['medium'].loc[tensor_idx],
    'F_fast': F_bands['fast'].loc[tensor_idx],
    
    # Bifurcation parameters
    'cusp_a': cusp_params['a'].loc[tensor_idx],
    'cusp_b': cusp_params['b'].loc[tensor_idx],
    'bimodality': cusp_params['bimodality'].loc[tensor_idx],
    
    # Proximity
    'saddle_dist': saddle_dist.loc[tensor_idx]
})

# Normalize each component
for col in fragility_tensor.columns:
    fragility_tensor[f'{col}_norm'] = (fragility_tensor[col] - fragility_tensor[col].min()) / \
                                       (fragility_tensor[col].max() - fragility_tensor[col].min() + 1e-8)

print(f'Fragility Tensor: {fragility_tensor.shape}')

In [ ]:
# === 4B: Integrated Fragility Index (IFI) from tensor ===

# Weights based on theory:
# - Slow band: structural buildup (early)
# - Cusp a < 0: bistability (danger)
# - Saddle dist low: near bifurcation

IFI = (
    0.20 * fragility_tensor['F_t_norm'] +
    0.25 * fragility_tensor['F_slow_norm'] +
    0.15 * fragility_tensor['F_medium_norm'] +
    0.10 * fragility_tensor['F_fast_norm'] +
    0.15 * (1 - fragility_tensor['cusp_a_norm']) +  # Inverted: low a = dangerous
    0.15 * (1 - fragility_tensor['saddle_dist_norm'])  # Inverted: low dist = dangerous
)

print(f'IFI range: {IFI.min():.3f} - {IFI.max():.3f}')

In [ ]:
# === 4C: Final visualization ===

fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# IFI
axes[0].fill_between(IFI.index, IFI.values, alpha=0.3, color='red')
axes[0].plot(IFI.index, IFI.values, 'r-', linewidth=0.8)
axes[0].axhline(IFI.quantile(0.8), color='orange', linestyle='--', label='Warning (80%)')
axes[0].axhline(IFI.quantile(0.95), color='darkred', linestyle='--', label='Critical (95%)')
axes[0].set_ylabel('IFI')
axes[0].set_title('Integrated Fragility Index (Formal Model)')
axes[0].legend()

# Temporal bands
axes[1].plot(fragility_tensor.index, fragility_tensor['F_slow'], 'b-', label='Slow', alpha=0.8)
axes[1].plot(fragility_tensor.index, fragility_tensor['F_medium'], 'orange', label='Medium', alpha=0.6)
axes[1].plot(fragility_tensor.index, fragility_tensor['F_fast'], 'r-', label='Fast', alpha=0.4)
axes[1].set_ylabel('Band Value')
axes[1].set_title('Temporal Decomposition')
axes[1].legend()

# Bifurcation status
axes[2].fill_between(cusp_params.index, cusp_params['a'], 0, 
                     where=cusp_params['a'] < 0, alpha=0.5, color='red', label='Bistable (a<0)')
axes[2].fill_between(cusp_params.index, cusp_params['a'], 0,
                     where=cusp_params['a'] >= 0, alpha=0.3, color='green', label='Stable (a≥0)')
axes[2].set_ylabel('Cusp a')
axes[2].set_title('Bifurcation Status')
axes[2].legend()

for ax in axes:
    for name, date in CRISES.items():
        if date > IFI.index.min() and date < IFI.index.max():
            ax.axvline(date, color='blue', alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/CARIA/models/formal_fragility.png', dpi=150)
plt.show()

In [ ]:
# === VALIDATION: Lead time per crisis ===
print('\n=== Lead Time Analysis (All Crises) ===')

def compute_lead_all(indicator, crisis_date, pre_days=180, threshold_pct=0.8):
    threshold = indicator.quantile(threshold_pct)
    pre = indicator[(indicator.index < crisis_date) & 
                    (indicator.index > crisis_date - pd.Timedelta(days=pre_days))]
    crossings = pre[pre > threshold]
    return (crisis_date - crossings.index[0]).days if len(crossings) > 0 else 0

lead_results = []
for crisis_name, crisis_date in CRISES.items():
    if crisis_date < IFI.index.min() + pd.Timedelta(days=180):
        continue
    if crisis_date > IFI.index.max():
        continue
    
    lead_ifi = compute_lead_all(IFI, crisis_date)
    lead_slow = compute_lead_all(fragility_tensor['F_slow_norm'], crisis_date)
    
    lead_results.append({
        'crisis': crisis_name,
        'date': crisis_date.date(),
        'IFI_lead': lead_ifi,
        'Slow_lead': lead_slow
    })

lead_df = pd.DataFrame(lead_results)
print(lead_df.to_string(index=False))
print(f'\nAverage lead: IFI={lead_df["IFI_lead"].mean():.0f}d, Slow={lead_df["Slow_lead"].mean():.0f}d')

In [ ]:
# === EXPORT ===
import json

export = {
    'version': 'Great Caria Formal Fragility v1.0',
    'generated': pd.Timestamp.now().isoformat(),
    'methodology': 'Factor Analysis + Wavelet Decomposition + Cusp Catastrophe',
    'current': {
        'ifi': float(IFI.iloc[-1]),
        'F_t': float(F_t.iloc[-1]),
        'F_slow': float(F_bands['slow'].iloc[-1]),
        'F_medium': float(F_bands['medium'].iloc[-1]),
        'F_fast': float(F_bands['fast'].iloc[-1]),
        'cusp_a': float(cusp_params['a'].iloc[-1]),
        'bifurcation_stable': bool(cusp_params['a'].iloc[-1] >= 0)
    },
    'factor_loadings': loadings.set_index('signal')['loading'].to_dict(),
    'thresholds': {
        'warning': float(IFI.quantile(0.8)),
        'critical': float(IFI.quantile(0.95))
    },
    'crises_validated': len(lead_results),
    'avg_lead_days': float(lead_df['IFI_lead'].mean()) if len(lead_df) > 0 else 0,
    'history': [
        {'date': d.isoformat(), 'ifi': float(IFI.loc[d])}
        for d in IFI.index[-252:]
    ]
}

with open('/content/drive/MyDrive/CARIA/models/formal_fragility_export.json', 'w') as f:
    json.dump(export, f, indent=2)

print('\n✓ Exported: formal_fragility_export.json')

In [ ]:
# === FINAL SUMMARY ===
print('\n' + '='*70)
print('GREAT CARIA - FORMAL FRAGILITY THEORY')
print('='*70)

print('\n📐 FORMAL MODEL:')
print('  F_t = FactorAnalysis(CF, Sync, ACF1, Var, Skew, Curvature)')
print(f'  Variance explained: {pca.explained_variance_ratio_[0]:.1%}')

print('\n⏱️ TEMPORAL BANDS:')
for band, leads in band_leads.items():
    print(f'  {band:8s}: avg lead = {np.mean(leads):.0f} days')

print('\n🌀 BIFURCATION DYNAMICS:')
print(f'  Cusp catastrophe model fitted')
print(f'  Current stability: {"STABLE" if cusp_params["a"].iloc[-1] >= 0 else "BISTABLE"}')

print('\n📊 CRISIS VALIDATION:')
print(f'  Crises tested: {len(lead_results)}')
print(f'  Average IFI lead: {lead_df["IFI_lead"].mean():.0f} days')

print('\n' + '='*70)